# Example Python code for rolling dice

In [14]:
import numpy as np
import random

## 1. Rolling a single 6-sided die many times and gathering the results

In [2]:
die_rolls = np.random.randint(1,7, size=100)
die_rolls.view()

array([1, 3, 4, 6, 4, 3, 6, 5, 4, 1, 3, 1, 1, 1, 1, 6, 1, 6, 1, 1, 4, 4,
       5, 5, 2, 3, 1, 2, 3, 6, 4, 3, 2, 5, 4, 4, 1, 4, 3, 6, 1, 4, 4, 2,
       4, 3, 2, 6, 6, 4, 4, 5, 6, 3, 2, 5, 5, 1, 2, 1, 6, 1, 3, 6, 6, 3,
       6, 3, 2, 3, 5, 3, 2, 1, 6, 5, 2, 5, 5, 4, 4, 3, 4, 2, 4, 3, 1, 1,
       6, 3, 6, 4, 2, 2, 2, 1, 1, 5, 1, 6])

In [3]:
number_of_threes = np.count_nonzero(die_rolls==3)
print(number_of_threes)

17


----
We can ask how the dice rolls are distributed.

In [4]:
distributions = {i:np.count_nonzero(die_rolls== i) for i in range(1,7)}
print(distributions)

{1: 21, 2: 14, 3: 17, 4: 19, 5: 12, 6: 17}


----
Or maybe how many odd rolls are followed by even rolls

In [5]:
num_odd_then_even = 0
for i in range(len(die_rolls)-1):
    if die_rolls[i]%2 == 1 and die_rolls[i+1]%2 == 0:
        num_odd_then_even += 1
print(num_odd_then_even)

30


## 2. Rolling Two 6-Sided Dice at Once

In [10]:
two_dice_rolls = np.random.randint(1,7, size=(100,2))
two_dice_rolls[:10]

array([[3, 5],
       [3, 3],
       [5, 6],
       [3, 3],
       [5, 3],
       [2, 3],
       [1, 6],
       [5, 4],
       [3, 4],
       [6, 4]])

----
Summing our two dice

In [11]:
summed_rolls = np.array([sum(row) for row in two_dice_rolls])
summed_rolls.view()

array([ 8,  6, 11,  6,  8,  5,  7,  9,  7, 10,  7, 12,  8,  5, 10,  9,  4,
        9, 11, 10,  7,  6,  9, 11,  4,  7,  4,  4,  7,  5, 10,  6,  6,  9,
        5, 11, 11,  4,  7,  9,  3,  6, 12,  9,  7, 11, 11,  5,  5,  8,  8,
        7,  7,  6,  5,  2,  8,  9,  8,  3, 10,  8,  6,  7,  6,  6,  6,  9,
        7,  7,  8,  2,  8, 12, 11,  5, 11, 11,  2,  8,  5, 11,  3,  9,  7,
        8,  3,  3,  6, 10,  7, 10,  5, 11,  3,  7, 10,  7,  5,  3])

In [12]:
sum_distributions = {i:np.count_nonzero(summed_rolls== i) for i in range(2,13)}
print(sum_distributions)

{2: 3, 3: 7, 4: 5, 5: 11, 6: 12, 7: 17, 8: 12, 9: 10, 10: 8, 11: 12, 12: 3}


## A Silly Game

This is going to be a very silly two player die rolling game. 

First to score 40 points wins. The first player is determined randomly.

On your turn
* Roll the die, you get the number of points as represented by the roll.
* You may choose to roll again. If you roll high than you last rolled, you double your points. If not, you instead gain no points this turn and your turn is over.
* You may choose to continue to roll again until your turn is over.

I will test two competing strategies.
* Roll once and walk away.
* Roll until you get a 6 or you bust.

---
Strategy 1: The player that just takes what they're given.  I'll call this the `scared_strat`.

In [ ]:
def scared_strat():
    return random.randint(1,6)

---
Strategy 2: The player that really likes to double things. I'll call this the `bold_strat`.

In [29]:
def bold_strat():
    roll = random.randint(1,6)
    score = roll
    while roll < 6:
        old_roll = roll
        roll = random.randint(1,6)
        if roll <= old_roll:
            return 0
        else:
            score *= 2
    return score

---
Creating a function that simulates games

In [79]:
def simulate_game(strat_list):
    '''
    Given two strategies for my goofy dice game, determine who wins.
    INPUT::
        - `strat_list`: a list of two functions that each output the points earned by a strategy in a given round
    OUTPUT::
        - the index of the winning strategy from the list
    '''
    points = [0,0] # Players start with 0 points
    who_first = random.randint(0,1) # Determine which player goes first
    who_second = (who_first+1)%2 # Use arithmetic to determine the index of the second player
    while True: # Each round of play is below. The fist person takes their turn and gets points.
        points[who_first] += strat_list[who_first]()
        if points[who_first] >= 40: # If the first player has scored enough, they win.
            return who_first
        else:
            points[who_second] += strat_list[who_second]() # Otherwise the second player goes.
        if points[who_second] >= 40: # If the second player scored enough, then they win.
            return who_second

---
Simulating the game 1000 times to get data on counts. 
There appears to be strong evidence that the bold strategy is the wrong one.

In [82]:
# Simulate 100 times and count how many times each strategy wins
wins = [simulate_game([scared_strat,bold_strat]) for i in range(1000)]
scared_wins = wins.count(0)
print(scared_wins)
bold_wins = wins.count(1)
print(bold_wins)

838
162
